<a href="https://colab.research.google.com/github/FeelOneE/Multi-doc-summarization/blob/main/multidoc_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#df = pd.DataFrame(result)
df = pd.read_csv('review_data.csv')
#df.to_csv('review_data.csv',sep=',', na_rep='NaN')
df.describe
df[['starScore','title']]

In [ ]:
# cleasing 함수 
import regex as re
import numpy as np
def cleasing(text):
    repl =''
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 자음, 모음 제거
    text = re.sub(pattern= pattern, repl=repl, string=text)
    #pattern = '[^\w\s]' # 특수기호 제거
    pattern = '[^가-히\s]' # 특수기호 제거
    text = re.sub(pattern= pattern, repl=repl, string=text)
    pattern = '<[^>]*>' # html 제거
    text = re.sub(pattern = pattern, repl='',string=text)
    return text
df['length'] = df.content.str.len()
df['label'] = np.where(df['starScore']<5, 0, 1)
df['clean_content'] = df['content'].map(lambda x: cleasing(x))
df2 = df[ df.length > 30 ] 
df2 = df2[['clean_content','label']]
df2[:10]

In [ ]:
df2['check_content'] = ''
df2

In [ ]:
from py_hanspell.hanspell import spell_checker
check_result = []

for index, item in df2.iterrows():
    try:
        df2['check_content'][index] = spell_checker.check(item['clean_content']).checked
    except Exception as e: 
        print("error occur! : ",e)
        print(item['clean_content'])
        continue

df2[:10]

In [ ]:
dataset_all = [ ['',0] for _ in range(len(df2)+1) ]
#print(dataset_all[0][0])
index = 0
for indexs, row in df2.iterrows():
    #print(index)
    dataset_all[index][0] = row['clean_content']
    dataset_all[index][1] = row['label']
    index = index+1

dataset_all[:5]

In [ ]:
import pandas as pd
df = pd.read_csv('train_data.csv')


In [ ]:
dataset_all = df.values.tolist()

In [ ]:
data_length = len(dataset_all)
train_length = int(data_length*0.6)

dataset_train = dataset_all[0:train_length]
dataset_test = dataset_all[train_length:data_length]
len(dataset_train),len(dataset_test)

(2931, 1954)

In [ ]:
!pip install mxnet --user
!pip install gluonnlp pandas tqdm --user
!pip install sentencepiece --user
!pip install transformers --user
!pip install torch --user

  Using cached mxnet-1.7.0.post2-py2.py3-none-win_amd64.whl (33.1 MB)
  Using cached numpy-1.16.6-cp39-cp39-win_amd64.whl
  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.3.3 requires numpy>=1.17, but you have numpy 1.16.6 which is incompatible.
tensorboard 2.4.1 requires requests<3,>=2.21.0, but you have requests 2.18.4 which is incompatible.
kobart 0.4 requires torch==1.7.1, but you have torch 1.8.1 which is incompatible.
You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.
  Using cached gluonnlp-0.10.0-cp39-cp39-win_amd64.whl
You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master 

  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' 'C:\Users\pil\AppData\Local\Temp\pip-req-build-jz7rgn_o'
You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to c:\users\pil\appdata\local\temp\pip-req-build-jz7rgn_o
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12769 sha256=bc5d67a71248ca405b1b10f12247eb9cf2f749a83410124a3f35f4868bc884c3
  Stored in directory: C:\Users\pil\AppData\Local\Temp\pip-ephem-wheel-cache-2773bmg2\wheels\0b\20\d8\031374f3d29b5150c59c814bed091fca7d6d4c8218148bf286
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4055632473211169911,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13474087696481933928
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4951408640
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 18075274555864177038
 physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18420437764985841757
 physical_device_desc: "device: XLA_GPU device"]

In [ ]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [ ]:
#!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
#!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [ ]:
#dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
#dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [ ]:
dataset_train[:5]

[['쑥 들어간 제품들을 평소에 좋아해서 쑥 화장품으로 대부분 쓰려고 해요 아침 저녁으로 매일 매일 쓰기에 부담이 없고 좋습니다자연약쑥 진정 토너라 자극없이 쓰기에도 좋고 녹차 루이보스 캐모마일 쑥 페퍼민트 로즈마리 유칼리투스잎이 들어간 그린 허브 성분이라 피부 진정에도 도움이 되는거 같아요',
  1.0],
 ['화장품 상자에 한번 더 뽁뽁이  포장으로  꼼꼼하게 담아져 있었고  화장품 용기는 플라스틱 용기인데  용량이 커서  한참 쓸 수 있어 좋네요 뚜껑을 열었을때  향기 좋네 하는 느낌이 없으니 향을 중요시  하는 분은  좋아하지  않을 듯 하네요바르고 조금 시간이 지나면  향이  없어지는 것  같아요용기에 적혀 있는 글씨가 너무 적어  안보였는데 세트 구입했더니 앰플인지 뭔지 모르겠는데  작은 샘플이 개가 있네요  싼 가격에  이 구성이면  좋네요',
  1.0],
 ['대용량에 헐 감동입니다 개는 지인도 팍팍 쓰라고 선물했어요이건 화장솜에 듬뿍 하셔서 많이 하셔도 흡수는 잘됩니다 저는 얼굴에 흡수가 안되는 타입인데 제품이 흡수 잘되고요 약간의 알콜냄새는 살짝 스치는 정도 입니다 코로나에 다들 든데 이 가격에 정말 감사합니다 마구 쓰시고 싶은 분은 구입 하시고 쫀질한 느낌과 깊은 느낌분들은 살짝 고민하시고 클렌징으로 쓰고 싶다 하시는 분들은 차 쓰시고 고가의 스킨을 바르시는것도 좋을듯 합니다',
  1.0],
 ['아직 사용전인데 오자마자 간병이모님이 머냐고 물어 보셔서 약쑥스킨이라고  하니 한번 보자고 해서 뚜껑 열어서 향 맞아보고 손에 조금만 부어 보라고 해서 부었더니 발라보시더니 금방 흡수되고 촉촉하다고 하시며 얼마냐고 물어 보셔서 써보시라고 그냥 하나 드렸네요 방에 환자도 뭔데 그리 크냐고 하시면서 어떻게 사냐고 물어보시고 다들 관심 보이네요 각질제거가 된다니 일단 닦토로 써보고 조으면 계속 사서 써보려구요',
  0.0],
 ['프레쥬 진정토너 여름에 시원하게 토너팩 하기 좋아요 향은 쑥 냄새보다는 시트러스계열 레몬향이 나는 편이구요대용량인데다

In [ ]:
dataset_test[:5]

[['향이 독하지않고 산뜻하니 좋습니다 이 가격에 피부에 도움을 바라는건 언감생심이죠 세안만으로 없어지지 않는 비비크림 잔여물이나 씻어주고 피부에 해롭지나 않았으면 더 바랄게 없네요',
  1.0],
 ['잘받았네요 사용전이지만 좋을거라 믿어요양이 많아 오래쓰겠네요', 1.0],
 ['산뜻하면서도  에센스같은 수분감이 있어요용량도 대용량이라 부담없이 쓸수있어 좋아요', 1.0],
 ['가격만큼은 하네요 여릉용 바디 로션 겸용으로 사용해야겠네요', 0.0],
 ['사용하는게 있어 아직 사용전이지만 상품평이 좋아 기대되네요', 1.0]]

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

TypeError: Argument 'text' has incorrect type (expected str, got float)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
data_train[0]

In [ ]:
data_test[0]

In [ ]:
from tqdm.notebook import tqdm

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))